In [4]:
import sys
import numpy as np
import os

sys.path.insert(0, '../')
import sys; sys.argv=['']; del sys
import matplotlib.pyplot as plt

from models import FullModel
from models import AutoEncoder
from models import train_full_model
from models import check_loaded_weight
from models import CallbackParams
from models import encoder, decoder
import lightgbm as lgb

Using TensorFlow backend.


In [5]:
RAW = True
if RAW:
    test_filename = 'test_raw.csv'
    data_dir = 'train_sampled_raw'
    from input_data import image_generator_xd_raw as image_generator_xd
    from input_data import df_to_image_array_xd_raw as df_to_image_array_xd
else:
    test_filename = 'test_simplified.csv'
    data_dir = 'train_sampled'
    from input_data import image_generator_xd
    from input_data import df_to_image_array_xd

# Setup

In [6]:
NCSVS = 100
SIZE = 128
BATCHSIZE = 400
BASESIZE = 256
NCATS = 340
IN_CHANNEL = 3
NUM_TOTAL_DATA = int(1020000*0.99)
STEPS = int(NUM_TOTAL_DATA/BATCHSIZE)
EPOCHS = 16
VERBOSE = 1
MODEL_PATH = '/data/doodle/h5'
INPUT_DIR = '/data/doodle/'

def f2cat(filename: str) -> str:
    return filename.split('.')[0]

def list_all_categories():
    files = os.listdir(os.path.join(INPUT_DIR, 'train'))
    return sorted([f2cat(f) for f in files], key=str.lower)

In [7]:
def apk(actual, predicted, k=3):
    """
    Source: https://github.com/benhamner/Metrics/blob/master/Python/ml_metrics/average_precision.py
    """
    if len(predicted) > k:
        predicted = predicted[:k]
    score = 0.0
    num_hits = 0.0
    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i + 1.0)
    if not actual:
        return 0.0
    return score / min(len(actual), k)

def mapk(actual, predicted, k=3):
    """
    Source: https://github.com/benhamner/Metrics/blob/master/Python/ml_metrics/average_precision.py
    """
    return np.mean([apk(a, p, k) for a, p in zip(actual, predicted)])

def preds2catids(predictions):
    return pd.DataFrame(np.argsort(-predictions, axis=1)[:, :3], columns=['a', 'b', 'c'])

def top_3_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=3)

# Ensemble

In [8]:
import os
import pandas as pd
prob_files = os.listdir('/data/doodle/prob/')
prob_files = list(map(lambda x: '/data/doodle/prob/'+x, prob_files))
prob = np.zeros((112199, 340))
features = ['prob_%d'%i for i in range(NCATS)]
prob_files

['/data/doodle/prob/prob_xception-115_3_400_0774.csv',
 '/data/doodle/prob/prob_inception_restnet_v2-128_3_500_0803.csv',
 '/data/doodle/prob/prob_inception_resnet_v2-128_3_500_0029.csv']

In [9]:
for f in prob_files:
    prob_df = pd.read_csv(f)
    prob += prob_df[features].values
    print('%s were processed' % f)
    key_id = prob_df.key_id
    del prob_df

/data/doodle/prob/prob_xception-115_3_400_0774.csv were processed
/data/doodle/prob/prob_inception_restnet_v2-128_3_500_0803.csv were processed
/data/doodle/prob/prob_inception_resnet_v2-128_3_500_0029.csv were processed


In [10]:
top3 = preds2catids(prob/len(prob_files))
top3.head()

,a,b,c
0,234,281,266
1,144,36,125
2,305,62,309
3,187,111,303
4,56,113,112


In [11]:
cat_dict = list_all_categories()
cat_dict = [cat.replace(' ', '_') for cat in cat_dict]

In [12]:
top3.loc[:,'a'] = list(map(lambda x: cat_dict[x], top3.a))
top3.loc[:,'b'] = list(map(lambda x: cat_dict[x], top3.b))
top3.loc[:,'c'] = list(map(lambda x: cat_dict[x], top3.c))

In [13]:
top3.loc[:, 'word'] = [' '.join((a, b, c)) for a, b, c in zip(top3.a, top3.b, top3.c)]
top3.head()

,a,b,c,word
0,radio,stereo,snorkel,radio stereo snorkel
1,hockey_puck,bottlecap,frying_pan,hockey_puck bottlecap frying_pan
2,The_Great_Wall_of_China,castle,toe,The_Great_Wall_of_China castle toe
3,mountain,finger,tent,mountain finger tent
4,campfire,fireplace,fire_hydrant,campfire fireplace fire_hydrant


In [14]:
submission_df = pd.concat([key_id, top3.word], axis=1)
submission_df.to_csv('/data/doodle/submission/ensembled_tmp.csv', index=False)

In [15]:
submission_df.head(3)

,key_id,word
0,9000003627287624,radio stereo snorkel
1,9000010688666847,hockey_puck bottlecap frying_pan
2,9000023642890129,The_Great_Wall_of_China castle toe
